# MACHINE ASSIGNEMENT PROBLEM

## Obiettivo e prerequisiti
Nella presente esercitazione verrà considerato un assignement problem, specifica tipologia di problema di programmazione lineare, basato sui seguenti presupposti:
- Minimizzazione del tempo associato al completamento di un certo numero di attività da parte di determinate risorse (uomo o macchinari)
- Assegnazione di un solo lavoro a una risorsa
- Assegnazione di un ogni risorsa ad un lavoro

Formalmente si può esprimere il problema come:

\begin{equation}
\text{Min} \quad Z = \sum_{(i)} \sum_{(j)} c_{i,j} x_{i,j}
\end{equation}

\begin{equation}
\text{subject to: }\quad \sum_{(i)} x_{i,j}=1
\end{equation}

\begin{equation}
\sum_{(j)} x_{i,j}=1
\end{equation}

\begin{equation}
x_{i,j} \in {0,1}
\end{equation}


Uno degli approcci matematici più diffusi per risolvere questa tipologia di problemi di assegnamento bilanciati è l'**algoritmo ungherese**.

Di seguito se ne illustra il funzionamento.

Data una matrice quadrata di ordine n rappresentante la matrice dei costi del problema di assegnamento, l'algoritmo si svolge come segue:

- Per ogni riga, individuare il minimo e sottrarlo a tutti gli elementi della riga;
- Per ogni colonna, individuare il minimo e sottrarlo a tutti gli elementi della colonna;
- Tutti gli zeri nella matrice devono essere coperti contrassegnando il minor numero possibile di righe e/o colonne;
- Se il numero minimo di linee necessarie è pari a n è possibile determinare un assegnamento ottimo altrimenti procedere con lo step successivo;
- Individuare l'elemento minimo (k) tra gli elementi non coperti da linee, sottrarlo agli elementi non coperti e sommarlo agli elementi che sono incrocio di due linee. Tornare allo step 3.

Risolvere questo problema quando il numero di lavori o di risorse cresce richiede specifiche tecniche computazionali.
A questo proposito in questa esercitazione si utilizzerà il pacchetto **Gurobi** di Python.

# DEFINIZIONE DEL PROBLEMA
L'azienda di riferimento possiede quattro macchine e quattro lavori da completare. Ogni macchina deve essere assegnata per completare un lavoro. Il tempo necessario (espresso in ore) per ogni macchina per completare ogni lavoro è mostrato nella tabella riportata di seguito. L'azienda ha l'interesse di ridurre al minimo il tempo totale necessario per completare i quattro lavori.

In [21]:
import pandas as pd
tabella = [[14, 5, 8, 7], [2, 12, 6, 5], [7, 8, 3, 9], [2, 4, 6, 10]]
pd.DataFrame(tabella, columns=["Job0","Job1", "Job2","Job3"])

,Job0,Job1,Job2,Job3
0,14,5,8,7
1,2,12,6,5
2,7,8,3,9
3,2,4,6,10


In [22]:
# Import della libreria gurobipy
from gurobipy import *

## Data management
Le 4 macchine e i 4 lavori costituiscono gli insiemi identificati dal problema

In [23]:
machines = ["M0","M1","M2", "M3"]
jobs = ["J0", "J1", "J2", "J3"]

I = range(len(machines))
J = range(len(jobs))

La matrice del problema indica il tempo necessario ad ogni macchina per eseguire ogni lavoro

In [24]:
Comp_job_time = [
    [14,5,8,7],
    [2,12,6,5],
    [7,8,3,9],
    [2,4,6,10]
]

## Definizione del modello

In [25]:
m = Model("Assignment Model")

## Definizione delle variabili
\begin{equation}
x_{ij} = 1 \text{ se la macchina i è assegnata al lavoro j}
\end{equation}
\begin{equation}
x_{ij} = 0 \text{ se la macchina i non è assegnata al lavoro j}
\end{equation}

In [26]:
X = {}
for i in I:
    for j in J:
       X[i,j] = m.addVar(vtype= GRB.BINARY)

## Funzione obiettivo
L'obiettivo è ridurre al minimo il tempo totale necessario per completare i quattro lavori.

In [27]:
m.setObjective(quicksum(Comp_job_time[i][j]*X[i,j] for i in I for j in J), GRB.MINIMIZE)

## Vincoli
Devono essere soddisfatti due vincoli:
- Ogni macchina esegue un lavoro forzando la sommatoria di X(i,j) su tutti i lavori ad essere uguale ad 1.
- Ogni lavoro deve essere completato da una macchina forzando la sommatoria di X(i,j) su tutte le macchine ad essere uguale ad 1.

Formalmente:

\begin{equation}
\sum_{j\in \text{J}}x_{i,j} = 1 \quad \forall  i \in I
\end{equation}

\begin{equation}
\sum_{i\in \text{I}}x_{i,j} = 1 \quad \forall  j \in J
\end{equation}


In [28]:
# Vincolo 1
for i in I:
     m.addConstr(quicksum(X[i,j] for j in J) == 1)
# Vincolo 2
for j in J:
    m.addConstr(quicksum(X[i,j] for i in I) == 1)

In [29]:
m.optimize()
print("Optimized time: ",m.objVal, "hours")

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 8 rows, 16 columns and 32 nonzeros
Model fingerprint: 0x0d9005ee
Variable types: 0 continuous, 16 integer (16 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 28.0000000
Presolve time: 0.00s
Presolved: 8 rows, 16 columns, 32 nonzeros
Variable types: 0 continuous, 16 integer (16 binary)

Root relaxation: objective 1.500000e+01, 6 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0      15.0000000   15.00000  0.00%     -    0s

Explored 1 nodes (6 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was

Sulla base delle ipotesi iniziali e dei risultati sottostanti, l'assegnazione ottimale di 15 ore può essere ottenuta assegnando:
- Macchina 0 al lavoro 1
- Macchina 1 al lavoro 3
- Macchina 2 al lavoro 2
- Macchina 3 al lavoro 0

In [30]:
for i in I:
    print("-"*30)
    print("Machine:",i)
    print("-"*30)   
    for j in J:
        print("   Job",j,X[i,j].x)

------------------------------
Machine: 0
------------------------------
   Job 0 -0.0
   Job 1 1.0
   Job 2 -0.0
   Job 3 -0.0
------------------------------
Machine: 1
------------------------------
   Job 0 0.0
   Job 1 -0.0
   Job 2 -0.0
   Job 3 1.0
------------------------------
Machine: 2
------------------------------
   Job 0 -0.0
   Job 1 -0.0
   Job 2 1.0
   Job 3 -0.0
------------------------------
Machine: 3
------------------------------
   Job 0 1.0
   Job 1 0.0
   Job 2 -0.0
   Job 3 -0.0


## Analisi dei risultati
Confrontando l'assegnazione delle macchine ai lavori riportata sopra con "Optimized time" calcolato dal modello di ottimizzazione è possibile dedurre che i risultati siano veritieri e soddisfacenti.